In [1]:
""" The main script for training

requirement: python 2, tensorflow r1.4 or r1.2 
"""
import os
import sys
import time
import math
import argparse
import pickle

import numpy as np
import tensorflow as tf
# from tensorflow.contrib.framework import nest

from data_loader import DataLoader
from model import Model

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True

In [3]:
def main():
  parser = argparse.ArgumentParser()
  parser.add_argument('--dim_rec', type=int, default=128,
                     help='size of RNN hidden state')
  parser.add_argument('--num_layers', type=int, default=2,
                     help='number of layers in the RNN. ')
  parser.add_argument('--batch_size', type=int, default=10,
                     help='minibatch size')
  parser.add_argument('--num_epochs', type=int, default=200,
                     help='number of epochs')
  parser.add_argument('--save_every', type=int, default=10,
                     help='save frequency by epoches')
  parser.add_argument('--model_dir', type=str, default='checkpoints',
                     help='directory to save model to')
  parser.add_argument('--summary_dir', type=str, default='summary',
                     help='directory to save tensorboard info')
  parser.add_argument('--max_grad_norm', type=float, default=1.,
                     help='clip gradients at this value')
  parser.add_argument('--learning_rate', type=float, default=0.001,
                     help='learning rate')
  parser.add_argument('--decay_rate', type=float, default=1.0,
                     help='decay rate for the optimizer')
  parser.add_argument('--num_mixture', type=int, default=2,
                     help='number of gaussian mixtures')
  parser.add_argument('--data_scale', type=float, default=1000,
                     help='factor to scale raw data down by')
  parser.add_argument('--load_model', type=str, default=None,
                     help='Reload a model checkpoint and restore training.' )
  parser.add_argument('--bptt_length', type=int, default=120,
                     help='How many steps should the gradients pass back.' )
  parser.add_argument('--loss_form', type=str, default='mse',
                     help='mse / gmm' )
  parser.add_argument('--constraint_factor', type=float, default=0.,
                     help='the weight for constraint term in the cost function.' )
  
  args = parser.parse_args(['--num_epochs','200'])

  args.num_epochs = 100
  args.save_every = 5


  train(args)


In [4]:
def train(args):
  data_loader = DataLoader(args.batch_size, args.data_scale, args.bptt_length)
  data_loader.reset_batch_pointer()

  if args.model_dir != '' and not os.path.exists(args.model_dir):
    os.makedirs(args.model_dir)

  with open(os.path.join(args.model_dir, 'config.pkl'), 'wb') as f:
    pickle.dump(args, f)
  print("hyperparam. saved.")

  model = Model(args)

  # training
  with tf.Session(config=config) as sess:

    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    if args.load_model is not None:
        saver.restore(sess, args.load_model)
        _, ep_start = args.load_model.rsplit("-", 1)
        ep_start = int(ep_start)
        model_steps = int(ep_start * data_loader.num_batches)
    else:
        ep_start = 0
        model_steps = last_model_steps = 0

    last_time = time.time()

    for ep in range(ep_start, args.num_epochs):
      ep_loss = []
      sess.run(tf.assign(model.lr, args.learning_rate * (args.decay_rate ** ep))) ###????????

      for i in range(int(data_loader.num_sequences / args.batch_size)):
        idx = ep * data_loader.num_sequences + i * args.batch_size
        start = time.time()
        x, y, w, c, lens = data_loader.next_batch()

        loss_list, model_steps = model.train(
          sess=sess, 
          sequence=x, 
          targets=y, 
          weights=w, 
          conditions=c, 
          subseq_length=args.bptt_length, 
          step_count=model_steps
          )

        ep_loss += loss_list
        ########?????????????????????
        if model_steps - last_model_steps >= 100:
          new_time = time.time()
          print(
            "Sequence %d/%d (epoch %d), batch %d, train_loss = %.3f, time/batch = %.3f" 
            % (
                idx,
                args.num_epochs * data_loader.num_sequences,
                ep,
                model_steps,
                np.mean(loss_list),
                (new_time - last_time) / (model_steps - last_model_steps)
              )
            )
          sys.stdout.flush()
          last_model_steps = model_steps
          last_time = new_time
      print("Epoch %d completed, average train loss %.6f, learning rate %.4f" % (ep, np.mean(ep_loss), args.learning_rate * (args.decay_rate ** ep)))
      sys.stdout.flush()
      if not os.path.isdir(args.model_dir):
        os.makedirs(args.model_dir)
      if (ep+1) % args.save_every == 0:
        checkpoint_path = os.path.join(args.model_dir, 'model.ckpt')
        saver.save(sess, save_path=checkpoint_path, global_step = (ep+1))
        print("model saved.")




In [5]:
#args.num_epochs = 4
#args.save_every = 2

if __name__ == "__main__":
  #with tf.device('/device:GPU:2'):
    main()

Maximal length of the training data is 169
Training data case distribution: [5348, 5108, 4904, 1208, 1260, 1224, 916, 1180, 1216, 1044, 876, 456, 1264, 992, 1080, 932, 1008, 420, 1160, 1036]
Validation data case distribution: [70, 67, 65, 15, 17, 16, 12, 15, 16, 14, 12, 6, 17, 13, 14, 13, 14, 5, 15, 13]
Shuffling training data...
Shuffling training data...
hyperparam. saved.
Number of trainable variables 6
[<tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(134, 512) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(256, 512) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>, <tf.Variable 'out_W:0' shape=(128, 3) dtype=float32_ref>, <tf.Variable 'out_b:0' shape=(3,) dtype=float32_ref>]
Sequence 970/3263200 (epoch 0), batch 100, train_loss = 4.186, time/batch = 0

Sequence 83384/3263200 (epoch 2), batch 8502, train_loss = 2.331, time/batch = 0.094
Sequence 84354/3263200 (epoch 2), batch 8602, train_loss = 1.765, time/batch = 0.093
Sequence 85304/3263200 (epoch 2), batch 8702, train_loss = 2.527, time/batch = 0.095
Sequence 86294/3263200 (epoch 2), batch 8802, train_loss = 2.301, time/batch = 0.096
Sequence 87284/3263200 (epoch 2), batch 8902, train_loss = 1.647, time/batch = 0.095
Sequence 88264/3263200 (epoch 2), batch 9002, train_loss = 2.033, time/batch = 0.093
Sequence 89254/3263200 (epoch 2), batch 9102, train_loss = 1.749, time/batch = 0.094
Sequence 90254/3263200 (epoch 2), batch 9202, train_loss = 1.141, time/batch = 0.095
Sequence 91224/3263200 (epoch 2), batch 9302, train_loss = 1.364, time/batch = 0.094
Sequence 92194/3263200 (epoch 2), batch 9402, train_loss = 1.182, time/batch = 0.095
Sequence 93194/3263200 (epoch 2), batch 9502, train_loss = 2.777, time/batch = 0.095
Sequence 94174/3263200 (epoch 2), batch 9602, train_loss = 1.545,

Sequence 173660/3263200 (epoch 5), batch 17704, train_loss = 1.255, time/batch = 0.095
Sequence 174630/3263200 (epoch 5), batch 17804, train_loss = 1.523, time/batch = 0.094
Sequence 175620/3263200 (epoch 5), batch 17904, train_loss = 3.230, time/batch = 0.094
Sequence 176610/3263200 (epoch 5), batch 18004, train_loss = 1.991, time/batch = 0.095
Sequence 177580/3263200 (epoch 5), batch 18104, train_loss = 0.933, time/batch = 0.094
Sequence 178570/3263200 (epoch 5), batch 18204, train_loss = 1.281, time/batch = 0.094
Sequence 179540/3263200 (epoch 5), batch 18304, train_loss = 0.804, time/batch = 0.094
Sequence 180510/3263200 (epoch 5), batch 18404, train_loss = 1.353, time/batch = 0.093
Sequence 181490/3263200 (epoch 5), batch 18504, train_loss = 1.549, time/batch = 0.094
Sequence 182450/3263200 (epoch 5), batch 18604, train_loss = 1.190, time/batch = 0.094
Sequence 183440/3263200 (epoch 5), batch 18704, train_loss = 2.450, time/batch = 0.095
Sequence 184410/3263200 (epoch 5), batch 18

Sequence 262906/3263200 (epoch 8), batch 26804, train_loss = 2.461, time/batch = 0.094
Sequence 263846/3263200 (epoch 8), batch 26904, train_loss = 1.945, time/batch = 0.094
Sequence 264826/3263200 (epoch 8), batch 27004, train_loss = 1.693, time/batch = 0.095
Sequence 265806/3263200 (epoch 8), batch 27104, train_loss = 2.122, time/batch = 0.096
Sequence 266776/3263200 (epoch 8), batch 27204, train_loss = 1.324, time/batch = 0.095
Sequence 267746/3263200 (epoch 8), batch 27304, train_loss = 1.973, time/batch = 0.095
Sequence 268726/3263200 (epoch 8), batch 27404, train_loss = 1.596, time/batch = 0.096
Sequence 269726/3263200 (epoch 8), batch 27504, train_loss = 3.005, time/batch = 0.095
Sequence 270706/3263200 (epoch 8), batch 27604, train_loss = 1.334, time/batch = 0.095
Sequence 271706/3263200 (epoch 8), batch 27704, train_loss = 3.468, time/batch = 0.096
Sequence 272686/3263200 (epoch 8), batch 27804, train_loss = 2.609, time/batch = 0.093
Sequence 273666/3263200 (epoch 8), batch 27

Sequence 353230/3263200 (epoch 10), batch 36004, train_loss = 1.687, time/batch = 0.094
Sequence 354200/3263200 (epoch 10), batch 36104, train_loss = 1.675, time/batch = 0.094
Sequence 355180/3263200 (epoch 10), batch 36204, train_loss = 1.826, time/batch = 0.089
Sequence 356140/3263200 (epoch 10), batch 36304, train_loss = 1.350, time/batch = 0.094
Sequence 357120/3263200 (epoch 10), batch 36404, train_loss = 1.363, time/batch = 0.095
Sequence 358090/3263200 (epoch 10), batch 36504, train_loss = 2.262, time/batch = 0.094
Epoch 10 completed, average train loss 1.572067, learning rate 0.0010
Shuffling training data...
Sequence 359062/3263200 (epoch 11), batch 36604, train_loss = 1.757, time/batch = 0.095
Sequence 360032/3263200 (epoch 11), batch 36704, train_loss = 1.901, time/batch = 0.094
Sequence 360992/3263200 (epoch 11), batch 36804, train_loss = 1.203, time/batch = 0.095
Sequence 361972/3263200 (epoch 11), batch 36904, train_loss = 1.321, time/batch = 0.094
Sequence 362952/3263200

...

Sequence 2880526/3263200 (epoch 88), batch 293646, train_loss = 1.084, time/batch = 0.092
Sequence 2881516/3263200 (epoch 88), batch 293746, train_loss = 1.207, time/batch = 0.093
Sequence 2882496/3263200 (epoch 88), batch 293846, train_loss = 0.972, time/batch = 0.092
Sequence 2883486/3263200 (epoch 88), batch 293946, train_loss = 1.230, time/batch = 0.095
Sequence 2884476/3263200 (epoch 88), batch 294046, train_loss = 1.348, time/batch = 0.092
Sequence 2885456/3263200 (epoch 88), batch 294146, train_loss = 0.985, time/batch = 0.092
Sequence 2886456/3263200 (epoch 88), batch 294246, train_loss = 1.193, time/batch = 0.093
Sequence 2887446/3263200 (epoch 88), batch 294346, train_loss = 0.873, time/batch = 0.094
Sequence 2888406/3263200 (epoch 88), batch 294446, train_loss = 1.312, time/batch = 0.093
Sequence 2889386/3263200 (epoch 88), batch 294546, train_loss = 1.709, time/batch = 0.093
Sequence 2890386/3263200 (epoch 88), batch 294647, train_loss = 0.634, time/batch = 0.092
Sequence 2

Sequence 2967790/3263200 (epoch 90), batch 302548, train_loss = 0.972, time/batch = 0.092
Sequence 2968750/3263200 (epoch 90), batch 302648, train_loss = 0.953, time/batch = 0.094
Epoch 90 completed, average train loss 1.151222, learning rate 0.0010
Shuffling training data...
Sequence 2969742/3263200 (epoch 91), batch 302748, train_loss = 0.901, time/batch = 0.091
Sequence 2970742/3263200 (epoch 91), batch 302848, train_loss = 0.964, time/batch = 0.092
Sequence 2971732/3263200 (epoch 91), batch 302948, train_loss = 0.546, time/batch = 0.092
Sequence 2972712/3263200 (epoch 91), batch 303048, train_loss = 0.915, time/batch = 0.093
Sequence 2973692/3263200 (epoch 91), batch 303148, train_loss = 1.316, time/batch = 0.092
Sequence 2974682/3263200 (epoch 91), batch 303248, train_loss = 0.904, time/batch = 0.093
Sequence 2975632/3263200 (epoch 91), batch 303348, train_loss = 1.056, time/batch = 0.091
Sequence 2976622/3263200 (epoch 91), batch 303448, train_loss = 1.367, time/batch = 0.094
Seq

Sequence 3054166/3263200 (epoch 93), batch 311350, train_loss = 1.566, time/batch = 0.094
Sequence 3055136/3263200 (epoch 93), batch 311450, train_loss = 0.973, time/batch = 0.093
Sequence 3056116/3263200 (epoch 93), batch 311550, train_loss = 1.133, time/batch = 0.093
Sequence 3057096/3263200 (epoch 93), batch 311650, train_loss = 0.959, time/batch = 0.094
Sequence 3058066/3263200 (epoch 93), batch 311750, train_loss = 0.800, time/batch = 0.092
Sequence 3059056/3263200 (epoch 93), batch 311850, train_loss = 2.476, time/batch = 0.093
Sequence 3060036/3263200 (epoch 93), batch 311950, train_loss = 1.191, time/batch = 0.092
Sequence 3061016/3263200 (epoch 93), batch 312050, train_loss = 1.280, time/batch = 0.093
Sequence 3061986/3263200 (epoch 93), batch 312150, train_loss = 1.142, time/batch = 0.092
Sequence 3062956/3263200 (epoch 93), batch 312250, train_loss = 1.399, time/batch = 0.092
Sequence 3063926/3263200 (epoch 93), batch 312350, train_loss = 0.997, time/batch = 0.092
Sequence 3

Sequence 3140452/3263200 (epoch 96), batch 320151, train_loss = 1.488, time/batch = 0.091
Sequence 3141432/3263200 (epoch 96), batch 320251, train_loss = 1.251, time/batch = 0.091
Sequence 3142412/3263200 (epoch 96), batch 320351, train_loss = 1.232, time/batch = 0.092
Sequence 3143382/3263200 (epoch 96), batch 320451, train_loss = 1.118, time/batch = 0.093
Sequence 3144352/3263200 (epoch 96), batch 320551, train_loss = 1.445, time/batch = 0.091
Sequence 3145332/3263200 (epoch 96), batch 320651, train_loss = 0.992, time/batch = 0.092
Sequence 3146322/3263200 (epoch 96), batch 320751, train_loss = 1.177, time/batch = 0.093
Sequence 3147312/3263200 (epoch 96), batch 320851, train_loss = 1.482, time/batch = 0.093
Sequence 3148302/3263200 (epoch 96), batch 320951, train_loss = 1.233, time/batch = 0.092
Sequence 3149302/3263200 (epoch 96), batch 321051, train_loss = 0.860, time/batch = 0.092
Sequence 3150282/3263200 (epoch 96), batch 321151, train_loss = 0.965, time/batch = 0.093
Sequence 3

Sequence 3227726/3263200 (epoch 98), batch 329053, train_loss = 0.987, time/batch = 0.092
Sequence 3228716/3263200 (epoch 98), batch 329153, train_loss = 0.897, time/batch = 0.093
Sequence 3229706/3263200 (epoch 98), batch 329253, train_loss = 1.545, time/batch = 0.093
Epoch 98 completed, average train loss 1.140088, learning rate 0.0010
Sequence 3230708/3263200 (epoch 99), batch 329353, train_loss = 1.225, time/batch = 0.093
Shuffling training data...
Sequence 3231688/3263200 (epoch 99), batch 329453, train_loss = 0.893, time/batch = 0.093
Sequence 3232668/3263200 (epoch 99), batch 329553, train_loss = 1.452, time/batch = 0.092
Sequence 3233638/3263200 (epoch 99), batch 329653, train_loss = 1.260, time/batch = 0.092
Sequence 3234618/3263200 (epoch 99), batch 329753, train_loss = 1.327, time/batch = 0.093
Sequence 3235618/3263200 (epoch 99), batch 329853, train_loss = 0.889, time/batch = 0.094
Sequence 3236578/3263200 (epoch 99), batch 329953, train_loss = 1.272, time/batch = 0.093
Seq

In [ ]:
#------------------------#
#  print training loss   #
#------------------------#

## 2. Alternative training method

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--dim_rec', type=int, default=128,
                     help='size of RNN hidden state')
parser.add_argument('--num_layers', type=int, default=2,
                     help='number of layers in the RNN. ')
parser.add_argument('--batch_size', type=int, default=10,
                     help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=200,
                     help='number of epochs')
parser.add_argument('--save_every', type=int, default=10,
                     help='save frequency by epoches')
parser.add_argument('--model_dir', type=str, default='checkpoints',
                     help='directory to save model to')
parser.add_argument('--summary_dir', type=str, default='summary',
                     help='directory to save tensorboard info')
parser.add_argument('--max_grad_norm', type=float, default=1.,
                     help='clip gradients at this value')
parser.add_argument('--learning_rate', type=float, default=0.001,
                     help='learning rate')
parser.add_argument('--decay_rate', type=float, default=1.0,
                     help='decay rate for the optimizer')
parser.add_argument('--num_mixture', type=int, default=2,
                     help='number of gaussian mixtures')
parser.add_argument('--data_scale', type=float, default=1000,
                     help='factor to scale raw data down by')
parser.add_argument('--load_model', type=str, default=None,
                     help='Reload a model checkpoint and restore training.' )
parser.add_argument('--bptt_length', type=int, default=120,
                     help='How many steps should the gradients pass back.' )
parser.add_argument('--loss_form', type=str, default='mse',
                     help='mse / gmm' )
parser.add_argument('--constraint_factor', type=float, default=0.,
                     help='the weight for constraint term in the cost function.' )
  
args = parser.parse_args(['--num_epochs','200'])

args.num_epochs = 4
args.save_every = 1


In [5]:
import sys
#reload(sys)
print(sys.path)

['', '/home/mye/tools/anacondas3/envs/tensorflow/lib/python36.zip', '/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6', '/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6/lib-dynload', '/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6/site-packages', '/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6/site-packages/Sphinx-1.5.6-py3.6.egg', '/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6/site-packages/svgwrite-1.1.6-py3.6.egg', '/home/mye/tools/anacondas3/envs/tensorflow/lib/python3.6/site-packages/IPython/extensions', '/home/mye/.ipython']


In [8]:
data_loader = DataLoader(args.batch_size, args.data_scale, args.bptt_length) # batch_size=10, bptt_length=120
data_loader.reset_batch_pointer()
for i in range(int(data_loader.num_sequences / args.batch_size)):
    x, y, w, c, lens = data_loader.next_batch()
    print('i:',i,' x.shape:', x.shape)

Maximal length of the training data is 169
Training data case distribution: [5348, 5108, 4904, 1208, 1260, 1224, 916, 1180, 1216, 1044, 876, 456, 1264, 992, 1080, 932, 1008, 420, 1160, 1036]
Validation data case distribution: [70, 67, 65, 15, 17, 16, 12, 15, 16, 14, 12, 6, 17, 13, 14, 13, 14, 5, 15, 13]
Shuffling training data...
Shuffling training data...
('i:', 0, ' x.shape:', (10, 120, 6))
('i:', 1, ' x.shape:', (10, 120, 6))
('i:', 2, ' x.shape:', (10, 120, 6))
('i:', 3, ' x.shape:', (10, 120, 6))
('i:', 4, ' x.shape:', (10, 120, 6))
('i:', 5, ' x.shape:', (10, 120, 6))
('i:', 6, ' x.shape:', (10, 120, 6))
('i:', 7, ' x.shape:', (10, 120, 6))
('i:', 8, ' x.shape:', (10, 120, 6))
('i:', 9, ' x.shape:', (10, 120, 6))
('i:', 10, ' x.shape:', (10, 120, 6))
('i:', 11, ' x.shape:', (10, 120, 6))
('i:', 12, ' x.shape:', (10, 120, 6))
('i:', 13, ' x.shape:', (10, 120, 6))
('i:', 14, ' x.shape:', (10, 120, 6))
('i:', 15, ' x.shape:', (10, 120, 6))
('i:', 16, ' x.shape:', (10, 120, 6))
('i:'

('i:', 406, ' x.shape:', (10, 120, 6))
('i:', 407, ' x.shape:', (10, 120, 6))
('i:', 408, ' x.shape:', (10, 120, 6))
('i:', 409, ' x.shape:', (10, 120, 6))
('i:', 410, ' x.shape:', (10, 120, 6))
('i:', 411, ' x.shape:', (10, 120, 6))
('i:', 412, ' x.shape:', (10, 120, 6))
('i:', 413, ' x.shape:', (10, 120, 6))
('i:', 414, ' x.shape:', (10, 120, 6))
('i:', 415, ' x.shape:', (10, 120, 6))
('i:', 416, ' x.shape:', (10, 120, 6))
('i:', 417, ' x.shape:', (10, 240, 6))
('i:', 418, ' x.shape:', (10, 120, 6))
('i:', 419, ' x.shape:', (10, 120, 6))
('i:', 420, ' x.shape:', (10, 120, 6))
('i:', 421, ' x.shape:', (10, 120, 6))
('i:', 422, ' x.shape:', (10, 120, 6))
('i:', 423, ' x.shape:', (10, 120, 6))
('i:', 424, ' x.shape:', (10, 120, 6))
('i:', 425, ' x.shape:', (10, 120, 6))
('i:', 426, ' x.shape:', (10, 120, 6))
('i:', 427, ' x.shape:', (10, 120, 6))
('i:', 428, ' x.shape:', (10, 120, 6))
('i:', 429, ' x.shape:', (10, 120, 6))
('i:', 430, ' x.shape:', (10, 120, 6))
('i:', 431, ' x.shape:', 

('i:', 979, ' x.shape:', (10, 120, 6))
('i:', 980, ' x.shape:', (10, 120, 6))
('i:', 981, ' x.shape:', (10, 120, 6))
('i:', 982, ' x.shape:', (10, 120, 6))
('i:', 983, ' x.shape:', (10, 120, 6))
('i:', 984, ' x.shape:', (10, 120, 6))
('i:', 985, ' x.shape:', (10, 120, 6))
('i:', 986, ' x.shape:', (10, 120, 6))
('i:', 987, ' x.shape:', (10, 120, 6))
('i:', 988, ' x.shape:', (10, 120, 6))
('i:', 989, ' x.shape:', (10, 120, 6))
('i:', 990, ' x.shape:', (10, 120, 6))
('i:', 991, ' x.shape:', (10, 120, 6))
('i:', 992, ' x.shape:', (10, 120, 6))
('i:', 993, ' x.shape:', (10, 120, 6))
('i:', 994, ' x.shape:', (10, 120, 6))
('i:', 995, ' x.shape:', (10, 120, 6))
('i:', 996, ' x.shape:', (10, 120, 6))
('i:', 997, ' x.shape:', (10, 120, 6))
('i:', 998, ' x.shape:', (10, 120, 6))
('i:', 999, ' x.shape:', (10, 120, 6))
('i:', 1000, ' x.shape:', (10, 120, 6))
('i:', 1001, ' x.shape:', (10, 120, 6))
('i:', 1002, ' x.shape:', (10, 120, 6))
('i:', 1003, ' x.shape:', (10, 120, 6))
('i:', 1004, ' x.shap

('i:', 1587, ' x.shape:', (10, 120, 6))
('i:', 1588, ' x.shape:', (10, 120, 6))
('i:', 1589, ' x.shape:', (10, 120, 6))
('i:', 1590, ' x.shape:', (10, 120, 6))
('i:', 1591, ' x.shape:', (10, 120, 6))
('i:', 1592, ' x.shape:', (10, 120, 6))
('i:', 1593, ' x.shape:', (10, 120, 6))
('i:', 1594, ' x.shape:', (10, 120, 6))
('i:', 1595, ' x.shape:', (10, 120, 6))
('i:', 1596, ' x.shape:', (10, 120, 6))
('i:', 1597, ' x.shape:', (10, 120, 6))
('i:', 1598, ' x.shape:', (10, 120, 6))
('i:', 1599, ' x.shape:', (10, 120, 6))
('i:', 1600, ' x.shape:', (10, 120, 6))
('i:', 1601, ' x.shape:', (10, 120, 6))
('i:', 1602, ' x.shape:', (10, 120, 6))
('i:', 1603, ' x.shape:', (10, 120, 6))
('i:', 1604, ' x.shape:', (10, 120, 6))
('i:', 1605, ' x.shape:', (10, 120, 6))
('i:', 1606, ' x.shape:', (10, 120, 6))
('i:', 1607, ' x.shape:', (10, 120, 6))
('i:', 1608, ' x.shape:', (10, 120, 6))
('i:', 1609, ' x.shape:', (10, 120, 6))
('i:', 1610, ' x.shape:', (10, 120, 6))
('i:', 1611, ' x.shape:', (10, 120, 6))


('i:', 2173, ' x.shape:', (10, 240, 6))
('i:', 2174, ' x.shape:', (10, 120, 6))
('i:', 2175, ' x.shape:', (10, 120, 6))
('i:', 2176, ' x.shape:', (10, 120, 6))
('i:', 2177, ' x.shape:', (10, 120, 6))
('i:', 2178, ' x.shape:', (10, 120, 6))
('i:', 2179, ' x.shape:', (10, 120, 6))
('i:', 2180, ' x.shape:', (10, 120, 6))
('i:', 2181, ' x.shape:', (10, 120, 6))
('i:', 2182, ' x.shape:', (10, 120, 6))
('i:', 2183, ' x.shape:', (10, 120, 6))
('i:', 2184, ' x.shape:', (10, 120, 6))
('i:', 2185, ' x.shape:', (10, 120, 6))
('i:', 2186, ' x.shape:', (10, 120, 6))
('i:', 2187, ' x.shape:', (10, 120, 6))
('i:', 2188, ' x.shape:', (10, 120, 6))
('i:', 2189, ' x.shape:', (10, 120, 6))
('i:', 2190, ' x.shape:', (10, 120, 6))
('i:', 2191, ' x.shape:', (10, 120, 6))
('i:', 2192, ' x.shape:', (10, 120, 6))
('i:', 2193, ' x.shape:', (10, 120, 6))
('i:', 2194, ' x.shape:', (10, 120, 6))
('i:', 2195, ' x.shape:', (10, 120, 6))
('i:', 2196, ' x.shape:', (10, 120, 6))
('i:', 2197, ' x.shape:', (10, 120, 6))


('i:', 2743, ' x.shape:', (10, 120, 6))
('i:', 2744, ' x.shape:', (10, 120, 6))
('i:', 2745, ' x.shape:', (10, 120, 6))
('i:', 2746, ' x.shape:', (10, 120, 6))
('i:', 2747, ' x.shape:', (10, 120, 6))
('i:', 2748, ' x.shape:', (10, 120, 6))
('i:', 2749, ' x.shape:', (10, 120, 6))
('i:', 2750, ' x.shape:', (10, 120, 6))
('i:', 2751, ' x.shape:', (10, 120, 6))
('i:', 2752, ' x.shape:', (10, 120, 6))
('i:', 2753, ' x.shape:', (10, 120, 6))
('i:', 2754, ' x.shape:', (10, 120, 6))
('i:', 2755, ' x.shape:', (10, 240, 6))
('i:', 2756, ' x.shape:', (10, 240, 6))
('i:', 2757, ' x.shape:', (10, 120, 6))
('i:', 2758, ' x.shape:', (10, 120, 6))
('i:', 2759, ' x.shape:', (10, 120, 6))
('i:', 2760, ' x.shape:', (10, 120, 6))
('i:', 2761, ' x.shape:', (10, 120, 6))
('i:', 2762, ' x.shape:', (10, 120, 6))
('i:', 2763, ' x.shape:', (10, 120, 6))
('i:', 2764, ' x.shape:', (10, 120, 6))
('i:', 2765, ' x.shape:', (10, 120, 6))
('i:', 2766, ' x.shape:', (10, 120, 6))
('i:', 2767, ' x.shape:', (10, 120, 6))


In [9]:
print(data_loader.num_sequences )

32632


In [7]:
noise_level = 0.001
print(np.random.normal(0, noise_level, 3))

[ 0.00105173  0.00144482 -0.00125339]


In [50]:
import random
def sample_cluster(pi):
      rand = random.random()
      print(rand)
      accumulate = 0.
      for i in range(len(pi)):
        accumulate += pi[i]
        print('iterations:',i)
        if accumulate >= rand:
          print('accumulate:',accumulate)
          return i
      raise ValueError("Cannot sample a cluster!")

In [69]:
sample_cluster([0.3, 0.7])

0.176895848302
('iterations:', 0)
('accumulate:', 0.3)


0

In [71]:
a = ['a', [1, 2]]
print(a[0])
print(a[1][-1])

a
2


In [ ]:
#m1:[batch*time, num_mixture]
mu = (m1, m2, m3)